# データセット作成

## GoogleNewsからトピックに関する記事を収集・トピック分類モデルの学習

In [1]:
import os
import re
import feedparser
import urllib
import requests
import json
import pprint
from bs4 import BeautifulSoup as bs
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import gensim
import pickle

In [2]:
# 検索ワード
TOPICS = {'COVID-19': 'コロナ', 'heartstroke': '熱中症', 'exercise': '運動', 'lifehack': 'ライフハック', 
          'mental': 'メンタルヘルス', 'meal': '食事', 'sleep': '睡眠'}

In [3]:
savedir = os.path.join('dataset', 'raw_csv')
os.makedirs(savedir, exist_ok=True)
data = []

for key in TOPICS:
    # 文字コードの関係で変換
    s_quote = urllib.parse.quote(TOPICS[key])
    url_b4 = 'https://news.google.com/search?q=' + s_quote + '&hl=ja&gl=JP&ceid=JP%3Aja'

    # 検索結果ページの情報を取得
    res = requests.get(url_b4)
    soup = bs(res.content, "html.parser")

    # すべての記事部の情報を選択
    articles = soup.select(".xrnccd")

    # 各記事の情報を取得
    news = []
    for i, entry in enumerate(articles, 1):
        # titleとsummaryをつなげたものを取得
        title = entry.find("h3").text
        summary = entry.find("span").text
        summary = title + "。" + summary

        news.append(summary)

    data.append(news)
    # データフレームに変換してcsvファイルで保存
    news_df = pd.DataFrame(news)
    print(news_df.head())  # 最初の5行を表示してデータ確認 
    filename = os.path.join(savedir, key + ".csv")
    news_df.to_csv(filename, encoding='utf-8-sig', index=False)

                                                   0
0  東京都 新型コロナ 新たに247人感染確認 200人超は4日連続。【NHK】東京都は29日、...
1  コロナ「第2波」、9府県で新規感染者が最大想定上回る [新型コロナウイルス]。厚生労働省は2...
2  神奈川 新型コロナ 106人感染確認 2人死亡。神奈川県内では29日、新型コロナウイルスに感...
3  吉岡秀隆 新型コロナに感染 - Yahoo!ニュース。俳優の吉岡秀隆（50）が29日、新型コ...
4  福岡県で男女3人死亡 新型コロナ 福岡市の会社でクラスター。福岡県では29日、新たに76人の...
                                                   0
0  熱中症の疑い 東京都内で50人が病院搬送。【NHK】東京都内では29日、熱中症の疑いで50人...
1  熱中症にならないために、牛乳を飲むべき理由とは（ダイヤモンド・オンライン）。暑い日が続き、連...
2  広い範囲で猛暑日の見込み 熱中症に警戒を。29日は東日本や西日本で猛烈な暑さとなり38度を超...
3  東日本や西日本中心 広範囲で猛烈な暑さ予想 熱中症に警戒を。【NHK】29日も東日本や西日本...
4  猛暑の週末 関東1都6県に熱中症警戒アラート。気象庁は、今日29日(土)を対象とした熱中症警...
                                                   0
0  運動不足の中高年 半年の有酸素運動で認知機能が向上｜ヘルスＵＰ｜NIKKEI。普段あまり運動...
1  独・ベルリンで“反マスク運動”３万８０００人デモ。ドイツでは、新型コロナウイルスの感染対策に...
2  被爆者運動こつこつと 機関紙「被団協」500号 1976年創刊（毎日新聞）。被爆者による唯一...
3  ソフトバンク中村晃、ピンクリボン運動の寄付金贈呈（日刊スポーツ）。ソフトバンク中村晃外野手（...
4  16歳少女を30人が列つくり性的暴行、イスラエル版「MeToo」運動に火（ＡＦＰ＝時事）。【...
                                              

#### 形態素解析のスクリプト

In [4]:
import MeCab
import pickle
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np


def listdir(path):
    return [f for f in os.listdir(path) if not f.startswith('.')]

def _split_to_words(text, to_stem=False):
    """
    入力: 'すべて自分のほうへ'
    出力: tuple(['すべて', '自分', 'の', 'ほう', 'へ'])
    """
    tagger = MeCab.Tagger('mecabrc')  # 別のTaggerを使ってもいい
    mecab_result = tagger.parse(text)
    info_of_words = mecab_result.split('\n')
    words = []
    for info in info_of_words:
        # macabで分けると、文の最後に’’が、その手前に'EOS'が来る
        if info == 'EOS' or info == '':
            break
            # info => 'な\t助詞,終助詞,*,*,*,*,な,ナ,ナ'
        info_elems = info.split('\t')
        part_of_speech = info_elems[4].split('-')
        # if part_of_speech in ['補助記号', '助詞']:
        if part_of_speech[0] != '名詞' or ('数詞' in part_of_speech):
            continue
        words.append(info_elems[0])
    return words

def words(text):
    words = _split_to_words(text=text, to_stem=False)
    return words


In [5]:
for i, d in enumerate(data):
    if i == 0:
        datas = d
    else:
        datas += d

In [6]:
words(datas[0])

['東京',
 '都',
 '新型',
 'コロナ',
 '感染',
 '確認',
 '連続',
 'NHK',
 '東京',
 '都',
 '都内',
 '新型',
 'コロナ',
 'ウイルス',
 '感染',
 'こと',
 '確認',
 '発表',
 '感染',
 '確認']

In [7]:
docs = []
for d in datas:
    docs.append(words(d))

#### 単語を収集・重要語の抽出

In [9]:
dct = gensim.corpora.Dictionary(docs)

unfiltered = dct.token2id.keys()
dct.filter_extremes(no_below=3, no_above=0.6)
filtered = dct.token2id.keys()
filtered_out = set(unfiltered) - set(filtered)
print("The following super common/rare words are filtered out...")
print(list(filtered_out), '\n')

print("Vocabulary after filtering...")
print(dct.token2id.keys(), "(%d words)" % len(dct.token2id.keys()), '\n')

The following super common/rare words are filtered out...
['両者', '幸一郎', '行財政', '鮎', 'ツイート', 'ハッシュ', '英文', 'プライベート', '転載', '一帯', '優先', '運行', '沐浴', '本日', '条例', 'woman', 'リクルート', '腹筋', 'あご', 'ペン', 'ＴＨＥ', '姫野', '平日', '公立', 'ブランケット', 'ホルダー', '林', '椅子', 'パール', '免疫', 'CA', '渡', '友', '湯崎', '再来', '加古川', '登山', '接触', '三佳', '説明', '図解', '泉', '旬', 'ロビンソン', 'カーネギー', '合わせ', '実効', '民友', '感情', 'ジオグラフィック', 'オープン', '政局', 'ベルト', '井戸', 'パブ', '中山', '追跡', '熊本', 'macOS', 'クリック', '宇部', '壁', '乗客', '古代', '花', '漁船', 'デビュー', '夕食', '妻', '解約', 'STARNEWS', 'ブレーク', '心拍', '持ち込み', '共有', 'ひととき', '特産', 'Underworld', '攻略', '学会', 'ビール', 'おかげ', 'カロ', 'サル', 'ジェイン', 'ホンダ', '本拠', 'バイオ', '因子', '靴下', '壮', '部屋づくり', 'リンカーン', 'リンク', '日大', '点滴', 'ソーシャル', '佐川', '早起き', '瞑想', '清水', 'ノンアルコール', 'クリップ', 'エジプト', '改正', 'レア', 'ポジティブ', '難民', 'トランス', '総裁', 'SALAD', 'TEL', '崩壊', '変換', '酒', '瞬時', 'ばあ', 'メートル', '対話', 'ペッパー', 'ホビー', 'サマー', 'home', 'ポップ', '紫苑', '自己', '調べ', '益城', 'collaborative', '構想', '再生', 'スペシャル', '漏洩', '建築', '青木', '増', '文学', 'トロント

#### Bag of Wordsの計算

In [11]:
def vec2dense(vec, num_terms):
    # Convert from sparse gensim format to dense list of numbers
    return list(gensim.matutils.corpus2dense([vec], num_terms=num_terms).T[0])

bow_docs = {}
bow_docs_all_zeros = {}
for i, d in enumerate(docs):
    sparse = dct.doc2bow(d)
    bow_docs[i] = sparse
    dense = vec2dense(sparse, num_terms=len(dct))
    # print(i, ":", dense)
    bow_docs_all_zeros[i] = all(d == 0 for d in dense)

print("\nall zeros...\n", [i for i in bow_docs_all_zeros if bow_docs_all_zeros[i]])


all zeros...
 []


#### ベクトル化，LSI Modelで次元削減

In [12]:
lsi_docs = {}
num_topics = len(TOPICS)
lsi_model = gensim.models.LsiModel(
    bow_docs.values(),
    id2word=dct,
    num_topics=num_topics)

for i in range(len(docs)):
    vec = bow_docs[i]
    sparse = lsi_model[vec]
    dense = vec2dense(sparse, num_topics)
    lsi_docs[i] = sparse
    print(i, ":", dense)

print("\nTopics")
print(lsi_model.print_topics())

0 : [4.6395516, -0.56001043, -0.65549403, 0.26457837, -0.08720266, 0.36275128, 0.26503327]
1 : [4.428692, -0.59183466, 0.105841674, 0.1385395, -0.36486655, -1.0607336, 0.24133326]
2 : [5.7852616, -1.0315067, -0.4710264, 0.23040138, -0.06464016, 0.010524315, 0.37316483]
3 : [3.4921246, -0.6374265, 0.049202096, 0.3027167, -0.39618406, -0.72536385, 0.21450283]
4 : [4.192374, -0.21804488, -0.17814787, -0.3842141, 0.2650114, 0.8897499, 0.6268911]
5 : [2.5688345, -0.11556018, 0.26144361, 0.48790345, -0.43765542, -0.97422916, 0.04172597]
6 : [4.59946, -0.56067115, -0.7311798, -0.30497786, 0.78842735, 1.8217603, -0.12767944]
7 : [4.3246984, -0.64180005, -0.5679658, -0.08919104, 0.34085912, 0.98642844, 0.33926922]
8 : [5.103632, -0.7774596, -0.4774051, 0.41893673, -0.3745744, -0.7996966, 0.25439692]
9 : [4.786503, -0.48718375, -0.7544284, -0.42146423, 0.9596559, 2.1863678, -0.2463874]
10 : [4.530458, -0.59835976, -0.65418375, -0.26878732, 0.6006564, 1.4926776, 0.18592319]
11 : [1.8686655, 0.156

#### Vector Normalization

In [13]:
unit_vecs = {}
for i in range(len(docs)):
    vec = vec2dense(lsi_docs[i], num_topics)
    unit_vec = vec / np.linalg.norm(vec, ord=2)
    unit_vecs[i] = unit_vec
#     print(i, ":", unit_vec)

#### Classification

In [14]:
all_data = [unit_vecs[i] for i in range(len(docs))]

all_labels = []
for i in range(len(TOPICS)):
    all_labels += [i]*100

train_data, test_data, train_label, test_label = train_test_split(all_data, all_labels)

# Train SVM classifier
classifier = SVC(probability=True)
classifier.fit(train_data, train_label)

/home/shun/.conda/envs/hack/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

#### 学習したモデルの精度を表示

In [15]:
# Prediction
predict_label = classifier.predict(test_data)

target_names = TOPICS.keys()
print(classification_report(test_label, predict_label, target_names=target_names))

              precision    recall  f1-score   support

    COVID-19       0.59      0.96      0.73        23
 heartstroke       0.96      1.00      0.98        27
    exercise       0.95      1.00      0.97        19
    lifehack       0.85      0.77      0.81        22
      mental       0.65      0.46      0.54        28
        meal       1.00      0.88      0.94        34
       sleep       0.95      0.86      0.90        22

   micro avg       0.84      0.84      0.84       175
   macro avg       0.85      0.85      0.84       175
weighted avg       0.85      0.84      0.84       175



In [16]:
# 例文
doc = words("東京都内では29日、熱中症の疑いで50人が病院に運ばれました。引き続き、冷房を適切に使用し、こまめに水分を取るなどの対策が必要です。")

dct_test = gensim.corpora.Dictionary.load('models/dict.dict')
lsi_model_test = gensim.models.LsiModel.load('models/lsi_model.model')
classifier_test = pickle.load(open('models/svm.pickle', 'rb'))

num_topics = len(TOPICS)
vec = dct_test.doc2bow(doc)
lsi_doc = lsi_model_test[vec]
vec = vec2dense(lsi_doc, num_topics)
unit_vec = vec / np.linalg.norm(vec, ord=2)
print(unit_vec)

prob = classifier_test.predict_proba([unit_vec])
print(prob)
print(f'Predicted label: {TOPICS[list(TOPICS.keys())[np.argmax(prob[0])]]}')

[ 0.12563999  0.7452729  -0.55416673 -0.06098475 -0.02584656 -0.3271385
 -0.10136777]
[[0.00124448 0.97228632 0.00274464 0.00186922 0.00595633 0.00192262
  0.01397639]]
Predicted label: 熱中症


#### モデルの保存

In [7]:
dct_test.save('models/dict.dict')

In [8]:
lsi_model_test.save('models/lsi_model.model')

In [9]:
pickle.dump(classifier_test, open('models/svm.pickle', 'wb'))